In [1]:
from IPython.display import display, clear_output, HTML
import time
import random
import ipywidgets as widgets
from jupyter_ui_poll import ui_events
import requests
from bs4 import BeautifulSoup
import json 
import pandas as pd

/var/folders/rf/h_plg61d4bvgmzmf0j7xbfgm0000gn/T/ipykernel_95317/86399730.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_event(btn):
    # display button description in output area
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

def register_text_input_event(text_input):
    event_info['type'] = "text_entry"
    event_info['description'] = text_input.value
    event_info['time'] = time.time()
    return


def text_input(prompt=None):
    text_input = widgets.Text(description=prompt, style= {'description_width': 'initial'})
    import warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    text_input.on_submit(register_text_input_event)
    display(text_input)
    event = wait_for_event(timeout=-1)
    text_input.disabled = True
    return event['description']

In [3]:
def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [4]:
def run_maths(difficulty):
    operator = ['+', '-']
    start_time = time.time()
    score = 0
    
    # Generate and display the starting number
    starting_num = random.randint(10 ** (difficulty - 1), 10 ** difficulty)
    html_outstartnum = HTML(f"""<h1>{starting_num}</h1>""")
    display(html_outstartnum)
    time.sleep(2)
    
    # Clear the output once after displaying the starting number
    clear_output(wait=False)

    equation = str(starting_num)

    # Generate equations and display them
    for _ in range(1 * difficulty): 
        sign = random.choice(operator)
        number = str(random.randint(10 ** (difficulty - 1), 10 ** difficulty - 1))
        correct_answer = eval(f"{equation}{sign}{number}")
        equation = correct_answer
        
        html_outsignnum = HTML(f"""<h1>{sign}{number}</h1>""")
        display(html_outsignnum)
        time.sleep(2)
        clear_output(wait=False)

    # Generate options for buttons, ensuring correct_answer is not included
    other_options = random.sample(range(correct_answer - 10, correct_answer + 11), 2)
    while correct_answer in other_options:
        other_options = random.sample(range(correct_answer - 10, correct_answer + 11), 2)
        
    # Create buttons with options
    btn1 = widgets.Button(description=str(other_options[0]))
    btn2 = widgets.Button(description=str(other_options[1]))
    btn3 = widgets.Button(description=str(correct_answer))
    
    # Assign event handlers to buttons
    btn1.on_click(register_event) 
    btn2.on_click(register_event) 
    btn3.on_click(register_event) 

    # Shuffle buttons
    buttons = [btn1, btn2, btn3]
    random.shuffle(buttons)   
    panel = widgets.HBox(buttons)
    display(panel)
    start_time1 = time.time()

    # Wait for user click event
    event_info = wait_for_event(timeout=5)

    # Process the event
    if event_info['description'] != "":
        print(f"User clicked: {event_info['description']}")
        if int(event_info['description']) == correct_answer:
            score = difficulty
            print(f"Well done! {score} points gained!")
            time.sleep(1)
        else:
            score = 0
            print("Incorrect, next question!")
            time.sleep(1)
    else:
        print("User did not click in time")
        score = 0

    end_time = time.time()
    time_taken = end_time - start_time
    response_time = end_time - start_time1

    print(f"You took {response_time:.2f} seconds")

    time.sleep(1)
    clear_output(wait=False)
    
    return response_time, time_taken, score


In [5]:
def run_mat(results_dict):
    
    print("Welcome to the Maths test")
    time.sleep(2)
    clear_output(wait=False)
    total_time = 0
    total_score1 = 0
    total_score2 = 0
    total_score3 = 0
    total_score = 0
    maximum_score1 = 0 
    maximum_score2 = 0 
    maximum_score3 = 0 
    maximum_score = 0
    
    while total_time < 180:
        if total_time < 60:
            difficulty = 1
        elif 60 <= total_time < 120:
            difficulty = 2
        else:
            difficulty = 3
    
        response_time, time_taken, score = run_maths(difficulty)
        total_time += time_taken
        if difficulty == 1:
            total_score1 += score
            maximum_score1 += difficulty
        if difficulty == 2:
            total_score2 += score
            maximum_score2 += difficulty
        if difficulty == 3:
            total_score3 += score
            maximum_score3 += difficulty

        results_dict['difficulty'].append(difficulty)
        results_dict['score'].append(score)
        results_dict['response_time'].append(response_time)

    total_score += total_score1 + total_score2 + total_score3
    maximum_score += maximum_score1 + maximum_score2 + maximum_score3 
    
    final_score1 = (total_score1/maximum_score1) * 100
    final_score2 = (total_score2/maximum_score2) * 100
    final_score3 = (total_score3/maximum_score3) * 100
    final_score = (total_score/maximum_score) * 100
    
    print("Test completed!")
    print(f"Final score: {final_score: .2f}%!")

    return final_score1, final_score2, final_score3, final_score

In [6]:
def run_full_test():
   
    random.seed(168)
    results_dict = {
        'difficulty': [],
        'score': [],
        'response_time': [],
    }
    
    id_instructions = """

    Enter your anonymised ID
    
    To generate an anonymous 4-letter unique user identifier please enter:
    - two letters based on the initials (first and last name) of a childhood friend
    - two letters based on the initials (first and last name) of a favourite actor / actress
    
    e.g. if your friend was called Charlie Brown and film star was Tom Cruise
         then your unique identifier would be CBTC
    """
    
    print(id_instructions)
    user_id = input()
    while len(user_id.strip()) == 0:
        print("Invalid input! User id cannot be empty.")
        user_id = input("Please enter your user id again: ")
    print("User entered id:", user_id)

    time.sleep(1)
    clear_output(wait=False)

    
    print("Please enter your age:")
    age = input()
    while len(str(age)) == 0:
        print("Invalid input! Age cannot be empty.")
        age = input("Please enter your age again: ")
    if float(age) < 0:
        print("Invalid input! Age cannot be negative.")
        age = input("Please enter your age again: ")
    print("User entered age:", age)

    time.sleep(1)
    clear_output(wait=False)
    
    print("What is your ethnicity?")
    
    # Create buttons with options
    btna1 = widgets.Button(description='White')
    btna2 = widgets.Button(description='Black or African American')
    btna3 = widgets.Button(description='Asian')
    btna4 = widgets.Button(description='Other')
    btna5 = widgets.Button(description='Rather not say')
    
    # Assign event handlers to buttons
    btna1.on_click(register_event) 
    btna2.on_click(register_event) 
    btna3.on_click(register_event) 
    btna4.on_click(register_event) 
    btna5.on_click(register_event) 

    display(btna1, btna2, btna3, btna4, btna5)
    event_infoa = wait_for_event(timeout=60)

    ethnicity = event_infoa['description']
    print(f"User clicked: {event_infoa['description']}")

    time.sleep(1)
    clear_output(wait=False)


    print("Please enter your biological sex:")
    
    btnb1 = widgets.Button(description='Male')
    btnb2 = widgets.Button(description='Female')
    btnb3 = widgets.Button(description='Rather not say')
    
    # Assign event handlers to buttons
    btnb1.on_click(register_event) 
    btnb2.on_click(register_event) 
    btnb3.on_click(register_event) 

    display(btnb1, btnb2, btnb3)
    event_infob = wait_for_event(timeout=60)

    sex = event_infob['description']
    print(f"User clicked: {event_infob['description']}")

    time.sleep(1)
    clear_output(wait=False)

    
    print("Did you have breakfast today:")
    btnc1 = widgets.Button(description='Yes')
    btnc2 = widgets.Button(description='No')
    btnc3 = widgets.Button(description='Rather not say')
    
    # Assign event handlers to buttons
    btnc1.on_click(register_event) 
    btnc2.on_click(register_event) 
    btnc3.on_click(register_event) 

    display(btnc1, btnc2, btnc3)
    event_infoc = wait_for_event(timeout=60)

    breakfast = event_infoc['description']
    print(f"User clicked: {event_infoc['description']}")

    time.sleep(1)
    clear_output(wait=False)
    
    print("How long did you sleep last night?")
    btnd1 = widgets.Button(description='less than 5 hours')
    btnd2 = widgets.Button(description='5-9 hours')
    btnd3 = widgets.Button(description='more than 9 hours')
    btnd4 = widgets.Button(description='Rather not say')
    
    # Assign event handlers to buttons
    btnd1.on_click(register_event) 
    btnd2.on_click(register_event) 
    btnd3.on_click(register_event) 
    btnd4.on_click(register_event) 

    display(btnd1, btnd2, btnd3, btnd4)
    event_infod = wait_for_event(timeout=60)

    sleep = event_infod['description']
    print(f"User clicked: {event_infod['description']}")

    time.sleep(1)
    clear_output(wait=False)

    
    final_score1, final_score2, final_score3, final_score = run_mat(results_dict)

    results_df = pd.DataFrame(results_dict)

    data_dict = {
    'userid': user_id,
    'age': age,
    'ethnicity': ethnicity,
    'sex': sex,
    'breakfast': breakfast,
    'sleep': sleep,
    'difficulty 1 score': final_score1,
    'difficulty 2 score': final_score2,
    'difficulty 3 score': final_score3,
    'total score': final_score,
    'results_json': results_df.to_json()
    }
    
    print("\nPlease read:")
    print("")
    print("We wish to record your response data to an anonymised public data repository. ")
    print("Your data will be used for educational teaching purposes practising data analysis and visualisation.")
    print("")
    print("Please type   yes   in the box below if you consent to the upload.")

    result = text_input("> ")
    
    print("User Entered", result)
    
    if result == "yes":
        print("Thanks for your participation. - your data will be uploaded.")
        print("Please contact a.fedorec@ucl.ac.uk if you have any questions or concerns regarding the stored results.")

        #send_to_google_form(data_dict, form_url)
        form_url = 'https://docs.google.com/forms/d/e/1FAIpQLScatG4tNxuQgUXNf1Kx7SuwWepLXMyShECc23JWCQ5B-aI6mQ/viewform'
        send_to_google_form(data_dict, form_url)
        
    else:
        print("No problem we hope you enjoyed the test.")

    return 

In [7]:
run_full_test()

Test completed!
Final score:  0.00%!

Please read:

We wish to record your response data to an anonymised public data repository. 
Your data will be used for educational teaching purposes practising data analysis and visualisation.

Please type   yes   in the box below if you consent to the upload.


Text(value='', description='> ', style=TextStyle(description_width='initial'))

User Entered yes
Thanks for your participation. - your data will be uploaded.
Please contact a.fedorec@ucl.ac.uk if you have any questions or concerns regarding the stored results.
